In [1]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram
from matplotlib import pyplot as plt
import speech_recognition as sr
from wit import Wit
from sklearn import preprocessing, model_selection

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GaussianNoise
from keras.optimizers import Adam

/home/marko/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def createPaddedSpectrogram(audio):
    f, t ,Sxx = spectrogram(audio[1], fs=44100)
    length = Sxx.shape[1] if Sxx.shape[1] < 450 else 450
    f_cut = f[f<2000.0]
    Sxx = Sxx[:12, :]
    Sxx_padded = np.zeros((12,450))
    Sxx_padded[:12, :length] = Sxx[:12,:length]
#     plt.figure(figsize=(18,5))
#     plt.pcolormesh(t,f_cut, Sxx)
#     plt.show()
    return Sxx_padded

In [3]:
def sayKeyword(keyword, output_file):
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    wit_api = Wit('VE3COSMD3FBL4DBGPHNZJICM7ZCUJ7J5')
    with mic as source:
        print("Adjusting mic...")
        recognizer.adjust_for_ambient_noise(mic, 3)
        print("Say water")
        audio = recognizer.listen(mic, 2, 2.5)
        print("Detecting what you said...")
        response = wit_api.speech(audio.get_wav_data(), None, {'Content-Type': 'audio/wav'})
        if response['_text'] != keyword:
            print('Please speak more clearly.')
        else:
            file = open(output_file,'wb')
            file.write(audio.get_wav_data())
            file.close()
            print("All good!")
    
    audio_data = wavfile.read(output_file)
    return audio_data

In [4]:
#adding water pronunciation spectrograms from various speakers
Sxx_array = np.empty((110,12,450))

for i in range (1,111):
    filename = "data/pronunciation_en_water({0}).wav".format(i)
    audio = wavfile.read(filename)  
    Sxx_padded = createPaddedSpectrogram(audio)
    Sxx_array[i-1] = Sxx_padded


In [5]:
#adding personal spectrogram to the array
#these are pre-recorded on Pre-record
for i in range(0,40):
    audio_data = wavfile.read('data/personal_{0}.wav'.format(i))
    Sxx_padded = createPaddedSpectrogram(audio_data)
    Sxx_array = np.append(Sxx_array, Sxx_padded)

In [6]:
Sxx_array = Sxx_array.reshape(150, 12, 450)
Y_array = np.zeros(150)
Y_array[110:] = 1

In [7]:
xtrain,xtest,ytrain,ytest = model_selection.train_test_split(Sxx_array,Y_array, train_size= 0.9, random_state = 7)

/home/marko/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
model = Sequential()
model.add(GaussianNoise(0.5, input_shape=(12, 450,)))
model.add(Dense(256))
model.add(Activation('softmax'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('softmax'))
model.add(Dense(128))
model.add(Dense(64))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer= 'nadam')
model.fit(xtrain, ytrain, batch_size=32, epochs=20, validation_data=(xtest,ytest))

Train on 135 samples, validate on 15 samples
Epoch 1/20
135/135 [==============================] - 1s 5ms/step - loss: 0.2426 - acc: 0.7111 - val_loss: 0.1172 - val_acc: 0.9333
Epoch 2/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2150 - acc: 0.7111 - val_loss: 0.0676 - val_acc: 0.9333
Epoch 3/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2149 - acc: 0.7111 - val_loss: 0.0812 - val_acc: 0.9333
Epoch 4/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2111 - acc: 0.7111 - val_loss: 0.1099 - val_acc: 0.9333
Epoch 5/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2069 - acc: 0.7111 - val_loss: 0.0805 - val_acc: 0.9333
Epoch 6/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2060 - acc: 0.7111 - val_loss: 0.1124 - val_acc: 0.9333
Epoch 7/20
135/135 [==============================] - 0s 2ms/step - loss: 0.2007 - acc: 0.7111 - val_loss: 0.1051 - val_acc: 0.9333
Epoch 8/20
135/135 [===========

In [9]:
#now trying recording test audio
test_data = sayKeyword('water', 'data/testAudio.wav')
test_padded = createPaddedSpectrogram(test_data)

Adjusting mic...
Say water
Detecting what you said...


ConnectionError: HTTPSConnectionPool(host='api.wit.ai', port=443): Max retries exceeded with url: /speech (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f57660340f0>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [ ]:
model.predict(test_padded.reshape(1,12,450))